# QUALITATIVE ANALYSIS
## TRIPADVISOR: HOTEL
* Esteban Ariza
* Johan Giraldo
* Mateo Valdes

## PREREQUISITES

### Install

In [ ]:
# %pip install textblob
# %pip install vanderSentiment
# %pip install nrclex
# %pip install pysentiment2
# %pip install pandas
# %pip install nltk
# %pip install nrclex
# %pip install vaderSentiment

### Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.util import ngrams
import json

import nltk
import pysentiment2 as ps

from nrclex import NRCLex
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import statistics

### Download

In [ ]:
# Required for NRCLex
nltk.download('punkt')
nltk.download('stopwords')

### Import CSVs

sustainable hotels and other hotels

In [ ]:
dir = os.path.abspath('')
SUSTAINABLE_HOTELS_FILE_PATH = '../data/exploratory_analysis/tripadvisor_hotels_sustainable_clean.csv'
OTHER_HOTELS_FILE_PATH = '../data/exploratory_analysis/tripadvisor_hotels_nonsustainable_clean.csv'

sustainable_csv_path = os.path.join(dir, SUSTAINABLE_HOTELS_FILE_PATH)
print(sustainable_csv_path)

others_csv_path = os.path.join(dir, OTHER_HOTELS_FILE_PATH)
print(others_csv_path)

sust_df = pd.read_csv(sustainable_csv_path)
others_df = pd.read_csv(others_csv_path)

## N-Grams

In [ ]:
sust_df['REVIEW_DATE'] = sust_df['REVIEW_DATE'].str.replace('Date of stay:', '')
others_df['REVIEW_DATE'] = others_df['REVIEW_DATE'].str.replace('Date of stay:', '')

Add the "TRIMESTER" column

In [ ]:
sust_df.REVIEW_DATE = pd.to_datetime(sust_df.REVIEW_DATE)
sust_df['TRIMESTER'] = pd.PeriodIndex(sust_df.REVIEW_DATE, freq="Q").strftime('%Y-Q%q')
sust_df['TRIMESTER'].head()

others_df.REVIEW_DATE = pd.to_datetime(others_df.REVIEW_DATE)
others_df['TRIMESTER'] = pd.PeriodIndex(others_df.REVIEW_DATE, freq="Q").strftime('%Y-Q%q')
others_df['TRIMESTER'].head()


In [ ]:
def get_ngram_counter(dataframe, n):
    dict = {}
    for tuple in dataframe.itertuples():
        # text = tuple['REVIEW_TEXT']
        text = tuple.REVIEW_TEXT
        # trimester = tuple['trimester']
        trimester = tuple.TRIMESTER
        tokens = nltk.word_tokenize(text)
        
        tokens = [w.lower() for w in tokens if w.isalpha()]
        tokens = [w for w in tokens if not w in stopwords.words('english')]
        ngram = ngrams(tokens, n)

        for item in ngram:
            word = " ".join(item)

            if trimester not in dict:
                dict[trimester] = Counter()    
            
            dict[trimester][word] += 1
    return dict

In [ ]:
sust_unigram_counter = get_ngram_counter(sust_df, 1)
sust_bigram_counter = get_ngram_counter(sust_df, 2)
sust_trigram_counter = get_ngram_counter(sust_df, 3)

In [ ]:
others_unigram_counter = get_ngram_counter(others_df, 1)
others_bigram_counter = get_ngram_counter(others_df, 2)
others_trigram_counter = get_ngram_counter(others_df, 3)

In [ ]:
print(others_unigram_counter.keys())

In [ ]:
with open("sust_unigram_counter.json", "w") as out:
    json.dump(sust_unigram_counter, out)

with open("sust_bigram_counter.json", "w") as out:
    json.dump(sust_bigram_counter, out)

with open("sust_trigram_counter.json", "w") as out:
    json.dump(sust_trigram_counter, out)


In [ ]:
with open("others_unigram_counter.json", "w") as out:
    json.dump(others_unigram_counter, out)

with open("others_bigram_counter.json", "w") as out:
    json.dump(others_bigram_counter, out)

with open("others_trigram_counter.json", "w") as out:
    json.dump(others_trigram_counter, out)

In [ ]:
for key, value in sust_unigram_counter.items():
    print(key)
    print(value.most_common(20))

for key, value in sust_bigram_counter.items():
    print(key)
    print(value.most_common(20))

for key, value in sust_trigram_counter.items():
    print(key)
    print(value.most_common(20))


In [ ]:
for key, value in others_unigram_counter.items():
    print(key)
    print(value.most_common(20))

for key, value in others_bigram_counter.items():
    print(key)
    print(value.most_common(20))

for key, value in others_trigram_counter.items():
    print(key)
    print(value.most_common(20))

## Sentiment

### Init

In [ ]:
# initialize the dictionaries
vader_analyzer = SentimentIntensityAnalyzer()
hiv4 = ps.HIV4()
lm = ps.LM()

# Create aux methods
def getPolarity(row):
    analysis = TextBlob(row)
    # print([analysis.sentiment.polarity, analysis.sentiment.subjectivity])
    return analysis.sentiment.polarity

def getSubjectivity(row):
    analysis = TextBlob(row)
    # print([analysis.sentiment.polarity, analysis.sentiment.subjectivity])
    return analysis.sentiment.polarity

In [21]:
hiv4.get_score(hiv4.tokenize('Im super happy to announce that im becoming a great father, because im actually a horrible father'))

{'Positive': 4,
 'Negative': 1,
 'Polarity': 0.599999880000024,
 'Subjectivity': 0.4545454132231443}

### First Iteration

#### Normal

get polarity and subjetivity from the review text

In [ ]:
sust_df['TB_polarity'] = sust_df['REVIEW_TEXT'].apply(lambda row : getPolarity(row))
sust_df['TB_subjectivity'] = sust_df['REVIEW_TEXT'].apply(lambda row: getSubjectivity(row))

others_df['TB_polarity'] = others_df['REVIEW_TEXT'].apply(lambda row : getPolarity(row))
others_df['TB_subjectivity'] = others_df['REVIEW_TEXT'].apply(lambda row: getSubjectivity(row))


others_df.head()

obtain the scores (vader, HIV4, LM, NRC)

In [ ]:
sust_df['vader_score'] = sust_df['REVIEW_TEXT'].apply(lambda row : vader_analyzer.polarity_scores(row))
others_df['vader_score'] = others_df['REVIEW_TEXT'].apply(lambda row : vader_analyzer.polarity_scores(row))

In [ ]:
sust_df['HIV4_score'] = sust_df['REVIEW_TEXT'].apply(lambda row: hiv4.get_score(hiv4.tokenize(row)))
others_df['HIV4_score'] = others_df['REVIEW_TEXT'].apply(lambda row: hiv4.get_score(hiv4.tokenize(row)))

In [ ]:
sust_df['LM_score'] = sust_df['REVIEW_TEXT'].apply(lambda row: lm.get_score(lm.tokenize(row)))
others_df['LM_score'] = others_df['REVIEW_TEXT'].apply(lambda row: lm.get_score(lm.tokenize(row)))

In [ ]:
sust_df['NRC_affect'] = sust_df['REVIEW_TEXT'].apply(lambda row: NRCLex(row).affect_frequencies)
others_df['NRC_affect'] = others_df['REVIEW_TEXT'].apply(lambda row: NRCLex(row).affect_frequencies)

In [ ]:
sust_df.head()

In [ ]:
sust_df = pd.concat([sust_df, sust_df['vader_score'].apply(pd.Series)], axis=1)
sust_df = pd.concat([sust_df, sust_df['LM_score'].apply(pd.Series).add_prefix('LM_')], axis=1)
sust_df = pd.concat([sust_df, sust_df['HIV4_score'].apply(pd.Series).add_prefix('HIV4_')], axis=1)

sust_df.head()

In [ ]:
others_df = pd.concat([others_df, others_df['vader_score'].apply(pd.Series)], axis=1)
others_df = pd.concat([others_df, others_df['LM_score'].apply(pd.Series).add_prefix('LM_')], axis=1)
others_df = pd.concat([others_df, others_df['HIV4_score'].apply(pd.Series).add_prefix('HIV4_')], axis=1)

others_df.head()

In [ ]:
sust_df.describe()

In [ ]:
others_df.describe()

Save df into a csv

In [ ]:
SUST_OUTPUT_PATH = '../data/qualitative_analysis/sentiment/tripadvisor_hotels_sustainable_sentiments.csv'
OTHER_OUTPUT_PATH = '../data/qualitative_analysis/sentiment/tripadvisor_hotels_nonsustainable_sentiments.csv'

sust_df.to_csv(SUST_OUTPUT_PATH, index=False)
others_df.to_csv(OTHER_OUTPUT_PATH, index=False)

#### Normalize

Import summaries

In [ ]:
SUMMARY_FILE_PATH = '../data/review_summarizer/summarized_reviews_by_year_and_hotel-small.csv'
summary_df = pd.read_csv(SUMMARY_FILE_PATH)

Normalize method

In [ ]:
def normalize(from_range, to_range, value):
    on_decimal = to_range[0] + ( (value - from_range[0]) * (to_range[1] - to_range[0]) / (from_range[1] - from_range[0]) )
    return round(on_decimal*2)/2

Vader

In [ ]:
vader_analyzer = SentimentIntensityAnalyzer()

def vaderRating(text):
    compound = vader_analyzer.polarity_scores(text)["compound"]
    return normalize((-1, 1), (1, 5), compound)

summary_df['VADER_RATING'] = summary_df['REVIEW_SUMMARY'].apply(vaderRating)

In [ ]:
print(summary_df['VADER_RATING'].value_counts())

HIV4

In [ ]:
hiv4 = ps.HIV4()

def hiv4Rating(text):
    polarity = hiv4.get_score(hiv4.tokenize(text))['Polarity']
    return normalize((-1, 1), (1, 5), polarity)

summary_df['HIV4_RATING'] = summary_df['REVIEW_SUMMARY'].apply(hiv4Rating)

In [ ]:
print(summary_df['HIV4_RATING'].value_counts())

LM

In [ ]:
lm = ps.LM()

def lmRating(text):
    polarity = lm.get_score(lm.tokenize(text))['Polarity']
    return normalize((-1, 1), (1, 5), polarity)

summary_df['LM_RATING'] = summary_df['REVIEW_SUMMARY'].apply(lmRating)

In [ ]:
print(summary_df['LM_RATING'].value_counts())

Overall

In [ ]:
def overallRating(row):
    return round( statistics.mean([row['VADER_RATING'], row['HIV4_RATING'], row['LM_RATING']])*2 )/2

summary_df['OVERALL_RATING'] = summary_df.apply(overallRating, axis=1)

In [ ]:
print(summary_df['OVERALL_RATING'].value_counts())

Summaries with rating

In [ ]:
summary_df

Some examples of reviews by rating

In [ ]:
print('\n1 bubble:', summary_df[summary_df['OVERALL_RATING'] == 1].iloc[0]['REVIEW_SUMMARY'])
print('\n2 bubbles:', summary_df[summary_df['OVERALL_RATING'] == 2].iloc[0]['REVIEW_SUMMARY'])
print('\n3 bubbles:', summary_df[summary_df['OVERALL_RATING'] == 3].iloc[0]['REVIEW_SUMMARY'])
print('\n4 bubbles:', summary_df[summary_df['OVERALL_RATING'] == 4].iloc[0]['REVIEW_SUMMARY'])
print('\n5 bubbles:', summary_df[summary_df['OVERALL_RATING'] == 5].iloc[0]['REVIEW_SUMMARY'])

Save df into a csv

In [ ]:
SUMMARY_OUTPUT_PATH = '../data/qualitative_analysis/sentiment/tripadvisor_hotels_summaries_sentiments-small.csv'
summary_df.to_csv(SUMMARY_OUTPUT_PATH, index=False)

In order to add the real hotel rating to the "summary_df" we need to import first the csv that we use to do the summaries.

In [ ]:
ORGINAL_SUST_HOTELS_FILE_PATH = '../data/exploratory_analysis/tripadvisor_hotels_sustainable_clean.csv'
ORIGINAL_OTHER_HOTELS_FILE_PATH = '../data/exploratory_analysis/tripadvisor_hotels_nonsustainable_clean.csv'

original_sust_df = pd.read_csv(ORGINAL_SUST_HOTELS_FILE_PATH)
original_other_df = pd.read_csv(ORIGINAL_OTHER_HOTELS_FILE_PATH)

original_df = pd.concat([original_sust_df, original_other_df], ignore_index=True)

Then, we add the actual hotel rating column

In [ ]:
hotel_df = original_df.groupby(["HOTEL_NAME", "HOTEL_RATING","HOTEL_LOCATION"]).size().reset_index(name='HOTEL_REVIEW_COUNT').drop(['HOTEL_REVIEW_COUNT'], axis=1)

join_df = summary_df.join(hotel_df.set_index('HOTEL_NAME'), on='HOTEL_NAME', validate='m:1')
join_df['HOTEL_RATING'] = join_df['HOTEL_RATING'].map(lambda x: x / 10)

In [ ]:
join_df

Density plot

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

ratings_types = [
    np.full(shape=join_df.shape[0], fill_value="OVERALL").tolist(),
    np.full(shape=join_df.shape[0], fill_value="REAL").tolist(),
    np.full(shape=join_df.shape[0], fill_value="VADER").tolist(),
    np.full(shape=join_df.shape[0], fill_value="HIV4").tolist(),
    np.full(shape=join_df.shape[0], fill_value="LM").tolist(),
]

rating_values = [
    join_df["OVERALL_RATING"].tolist(),
    join_df["HOTEL_RATING"].tolist(),
    join_df["VADER_RATING"].tolist(),
    join_df["HIV4_RATING"].tolist(),
    join_df["LM_RATING"].tolist(),
]



ratings_df = pd.DataFrame(data={
    "TYPE":flatten(ratings_types),
    "RATING":flatten(rating_values)
})

ratings_df

sns.kdeplot(
    data=ratings_df,
    x="RATING",
    hue="TYPE",
    # bw_adjust=4
)

Rating stats

In [ ]:
join_df.loc[:, join_df.columns!='REVIEW_DATE'].describe()
# join_df.median()

Save "join_df" into a csv

In [ ]:
WHR_OUTPUT_PATH = '../data/qualitative_analysis/sentiment/tripadvisor_hotels_summaries_whr_sentiments-small.csv'
join_df.to_csv(WHR_OUTPUT_PATH, index=False)

### Second Iteration

In [ ]:
#Import datasets [Disclaimer: Get the path to the actual file]
df_3s = pd.read_csv("../data/Iter2Lda/tripadvisor_hotels_3_clean.csv") # 3 Stars Hotels
df_5sy = pd.read_csv("../data/Iter2Lda/tripadvisor_hotels_sustainable_clean_5stars.csv") # 5 Stars Sustainable Hotels
df_5sn = pd.read_csv("../data/Iter2Lda/tripadvisor_hotels_nonsustainable_clean_5stars.csv") # 5 Stars Non-Sustainable Hotels

#### Dataset 1 (5 Stars Sustainable Hotels + 3 Stars Hotels)

In [ ]:
# Concat dfs
df_d1 = pd.concat([df_3s, df_5sy])
# Polarity
df_d1['TB_polarity'] = df_d1['REVIEW_TEXT'].apply(lambda row : getPolarity(row))
# Subjectivity
df_d1['TB_subjectivity'] = df_d1['REVIEW_TEXT'].apply(lambda row: getSubjectivity(row))
# VADER
df_d1['vader_score'] = df_d1['REVIEW_TEXT'].apply(lambda row : vader_analyzer.polarity_scores(row))
# HIV4
df_d1['HIV4_score'] = df_d1['REVIEW_TEXT'].apply(lambda row: hiv4.get_score(hiv4.tokenize(row)))
# LM
df_d1['LM_score'] = df_d1['REVIEW_TEXT'].apply(lambda row: lm.get_score(lm.tokenize(row)))
# NRC
df_d1['NRC_affect'] = df_d1['REVIEW_TEXT'].apply(lambda row: NRCLex(row).affect_frequencies)

# Format dictionaries results
df_d1 = pd.concat([
    df_d1
    ,df_d1['vader_score'].apply(pd.Series).add_prefix('VADER_')
    ,df_d1['HIV4_score'].apply(pd.Series).add_prefix('HIV4_')
    ,df_d1['LM_score'].apply(pd.Series).add_prefix('LM_')
    ,df_d1['NRC_affect'].apply(pd.Series).add_prefix('NRC_')
], axis=1)

# Drop unnecesary columns
df_d1 = df_d1.drop(columns=['vader_score','HIV4_score','LM_score','NRC_affect'])

df_d1.head()

In [ ]:
# Save data
df_d1.to_csv("../data/sentiment/sentiment_d1.csv",index=False)

In [ ]:
# Analysis
df_d1_l = pd.read_csv("../data/sentiment/sentiment_d1.csv")
df_d1_l.describe()
df_d1_l.columns

In [35]:
# Add year column
df_d1_l['REVIEW_YEAR'] = df_d1_l['REVIEW_DATE'].apply(lambda x: int(str(x).split('-')[0]))

# Group by hotel and year
df_d1_lg = df_d1_l[
    [
        'HOTEL_NAME','REVIEW_YEAR',
        'TB_polarity', 'TB_subjectivity',
        # 'VADER_neg', 'VADER_neu', 'VADER_pos',
        'VADER_compound',
        # 'HIV4_Positive', 'HIV4_Negative',
        'HIV4_Polarity','HIV4_Subjectivity',
        # 'LM_Positive', 'LM_Negative',
        'LM_Polarity', 'LM_Subjectivity',
        'NRC_fear', 'NRC_anger', 'NRC_anticip', 'NRC_trust','NRC_surprise', 'NRC_positive', 'NRC_negative', 'NRC_sadness','NRC_disgust', 'NRC_joy', 'NRC_anticipation'
    ]
].groupby(['HOTEL_NAME','REVIEW_YEAR']).mean()

# Save data
df_d1_lg.to_csv("../data/sentiment/sentiment_hy_d1.csv")

#### Dataset 2 (5 Stars Non Sustainable Hotels + 3 Stars Hotels)

In [ ]:
# Concat dfs
df_d2 = pd.concat([df_3s, df_5sn])

# Polarity
df_d2['TB_polarity'] = df_d2['REVIEW_TEXT'].apply(lambda row : getPolarity(row))
# Subjectivity
df_d2['TB_subjectivity'] = df_d2['REVIEW_TEXT'].apply(lambda row: getSubjectivity(row))
# VADER
df_d2['vader_score'] = df_d2['REVIEW_TEXT'].apply(lambda row : vader_analyzer.polarity_scores(row))
# HIV4
df_d2['HIV4_score'] = df_d2['REVIEW_TEXT'].apply(lambda row: hiv4.get_score(hiv4.tokenize(row)))
# LM
df_d2['LM_score'] = df_d2['REVIEW_TEXT'].apply(lambda row: lm.get_score(lm.tokenize(row)))
# NRC
df_d2['NRC_affect'] = df_d2['REVIEW_TEXT'].apply(lambda row: NRCLex(row).affect_frequencies)

# Format dictionaries results
df_d2 = pd.concat([
    df_d2
    ,df_d2['vader_score'].apply(pd.Series).add_prefix('VADER_')
    ,df_d2['HIV4_score'].apply(pd.Series).add_prefix('HIV4_')
    ,df_d2['LM_score'].apply(pd.Series).add_prefix('LM_')
    ,df_d2['NRC_affect'].apply(pd.Series).add_prefix('NRC_')
], axis=1)

# Drop unnecesary columns
df_d2 = df_d2.drop(columns=['vader_score','HIV4_score','LM_score','NRC_affect'])

df_d2.head()

In [ ]:
# Save data
df_d2.to_csv("../data/sentiment/sentiment_d2.csv",index=False)

In [36]:
# Analysis
df_d2_l = pd.read_csv("../data/sentiment/sentiment_d2.csv")
df_d2_l.describe()

C:\Users\ariza\AppData\Local\Temp\ipykernel_24988\1664337518.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_d2_l = pd.read_csv("../data/sentiment/sentiment_d2.csv")


,HOTEL_RATING,HOTEL_PRICE,REVIEW_RATING,REVIEW_HELPFUL_VOTES,TB_polarity,TB_subjectivity,VADER_neg,VADER_neu,VADER_pos,VADER_compound,...,NRC_anger,NRC_anticip,NRC_trust,NRC_surprise,NRC_positive,NRC_negative,NRC_sadness,NRC_disgust,NRC_joy,NRC_anticipation
count,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,...,136667.000000,136667.0,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,136667.000000,121532.000000
mean,45.373133,219.153475,4.548004,1.298002,0.332626,0.332626,0.021958,0.742179,0.235862,0.861698,...,0.018988,0.0,0.178173,0.060543,0.332822,0.059390,0.034864,0.010431,0.170320,0.131198
std,3.590084,287.566380,0.831729,1.760918,0.162276,0.162276,0.032937,0.095590,0.104159,0.322031,...,0.035038,0.0,0.072250,0.052853,0.110458,0.073977,0.046416,0.026467,0.068797,0.061436
min,30.000000,0.000000,1.000000,0.000000,-1.000000,-1.000000,0.000000,0.144000,0.000000,-0.995700,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393
25%,45.000000,76.000000,4.000000,1.000000,0.233886,0.233886,0.000000,0.682000,0.162000,0.911800,...,0.000000,0.0,0.137931,0.000000,0.268293,0.000000,0.000000,0.000000,0.133333,0.090909
50%,45.000000,140.000000,5.000000,1.000000,0.333431,0.333431,0.008000,0.750000,0.229000,0.963300,...,0.000000,0.0,0.181818,0.058824,0.322581,0.043478,0.000000,0.000000,0.175000,0.125000
75%,50.000000,252.000000,5.000000,1.000000,0.434259,0.434259,0.034000,0.811000,0.302000,0.981700,...,0.033333,0.0,0.222222,0.093750,0.384615,0.090909,0.058824,0.000000,0.214286,0.160000
max,50.000000,3001.000000,5.000000,209.000000,1.000000,1.000000,0.441000,1.000000,0.856000,0.999700,...,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,1.000000,1.000000


In [37]:
# Add year column
df_d2_l['REVIEW_YEAR'] = df_d2_l['REVIEW_DATE'].apply(lambda x: int(str(x).split('-')[0]))

# Group by hotel and year
df_d2_lg = df_d2_l[
    [
        'HOTEL_NAME','REVIEW_YEAR',
        'TB_polarity', 'TB_subjectivity',
        # 'VADER_neg', 'VADER_neu', 'VADER_pos',
        'VADER_compound',
        # 'HIV4_Positive', 'HIV4_Negative',
        'HIV4_Polarity','HIV4_Subjectivity',
        # 'LM_Positive', 'LM_Negative',
        'LM_Polarity', 'LM_Subjectivity',
        'NRC_fear', 'NRC_anger', 'NRC_anticip', 'NRC_trust','NRC_surprise', 'NRC_positive', 'NRC_negative', 'NRC_sadness','NRC_disgust', 'NRC_joy', 'NRC_anticipation'
    ]
].groupby(['HOTEL_NAME','REVIEW_YEAR']).mean()

# Save data
df_d2_lg.to_csv("../data/sentiment/sentiment_hy_d2.csv")

#### Dataset 3 (3 Stars Hotels)

In [ ]:
# Concat dfs
df_d3 = df_3s

# Polarity
df_d3['TB_polarity'] = df_d3['REVIEW_TEXT'].apply(lambda row : getPolarity(row))
# Subjectivity
df_d3['TB_subjectivity'] = df_d3['REVIEW_TEXT'].apply(lambda row: getSubjectivity(row))
# VADER
df_d3['vader_score'] = df_d3['REVIEW_TEXT'].apply(lambda row : vader_analyzer.polarity_scores(row))
# HIV4
df_d3['HIV4_score'] = df_d3['REVIEW_TEXT'].apply(lambda row: hiv4.get_score(hiv4.tokenize(row)))
# LM
df_d3['LM_score'] = df_d3['REVIEW_TEXT'].apply(lambda row: lm.get_score(lm.tokenize(row)))
# NRC
df_d3['NRC_affect'] = df_d3['REVIEW_TEXT'].apply(lambda row: NRCLex(row).affect_frequencies)

# Format dictionaries results
df_d3 = pd.concat([
    df_d3
    ,df_d3['vader_score'].apply(pd.Series).add_prefix('VADER_')
    ,df_d3['HIV4_score'].apply(pd.Series).add_prefix('HIV4_')
    ,df_d3['LM_score'].apply(pd.Series).add_prefix('LM_')
    ,df_d3['NRC_affect'].apply(pd.Series).add_prefix('NRC_')
], axis=1)

# Drop unnecesary columns
df_d3 = df_d3.drop(columns=['vader_score','HIV4_score','LM_score','NRC_affect'])

df_d3.head()

In [ ]:
# Save data
df_d3.to_csv("../data/sentiment/sentiment_d3.csv",index=False)

In [38]:
# Analysis
df_d3_l = pd.read_csv("../data/sentiment/sentiment_d3.csv")
df_d3_l.describe()

,HOTEL_RATING,HOTEL_PRICE,REVIEW_RATING,REVIEW_HELPFUL_VOTES,TB_polarity,TB_subjectivity,VADER_neg,VADER_neu,VADER_pos,VADER_compound,...,NRC_anger,NRC_anticip,NRC_trust,NRC_surprise,NRC_positive,NRC_negative,NRC_sadness,NRC_disgust,NRC_joy,NRC_anticipation
count,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,...,68468.000000,68468.0,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,68468.000000,61022.000000
mean,44.643483,93.512955,4.469197,1.182552,0.315818,0.315818,0.022451,0.744209,0.233338,0.860392,...,0.019693,0.0,0.182975,0.059481,0.327046,0.061598,0.036102,0.009907,0.168529,0.131067
std,3.927921,90.284917,0.840206,1.538144,0.154645,0.154645,0.032851,0.095521,0.103767,0.320219,...,0.035836,0.0,0.070610,0.052227,0.108826,0.076739,0.047461,0.026528,0.066396,0.060830
min,30.000000,0.000000,1.000000,0.000000,-1.000000,-1.000000,0.000000,0.238000,0.000000,-0.989600,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857
25%,45.000000,0.000000,4.000000,1.000000,0.220949,0.220949,0.000000,0.685000,0.159000,0.909200,...,0.000000,0.0,0.142857,0.000000,0.263158,0.000000,0.000000,0.000000,0.133333,0.090909
50%,45.000000,94.000000,5.000000,1.000000,0.316536,0.316536,0.010000,0.752000,0.225000,0.961800,...,0.000000,0.0,0.185185,0.058824,0.315789,0.045455,0.022222,0.000000,0.173913,0.125000
75%,45.000000,135.000000,5.000000,1.000000,0.412500,0.412500,0.034000,0.813000,0.299000,0.981100,...,0.034483,0.0,0.222222,0.090909,0.375000,0.093750,0.062500,0.000000,0.210526,0.160000
max,50.000000,432.000000,5.000000,67.000000,1.000000,1.000000,0.427000,1.000000,0.762000,0.999600,...,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,1.000000,1.000000


In [40]:
# Add year column
df_d3_l['REVIEW_YEAR'] = df_d3_l['REVIEW_DATE'].apply(lambda x: int(str(x).split('-')[0]))

# Group by hotel and year
df_d3_lg = df_d3_l[
    [
        'HOTEL_NAME','REVIEW_YEAR',
        'TB_polarity', 'TB_subjectivity',
        # 'VADER_neg', 'VADER_neu', 'VADER_pos',
        'VADER_compound',
        # 'HIV4_Positive', 'HIV4_Negative',
        'HIV4_Polarity','HIV4_Subjectivity',
        # 'LM_Positive', 'LM_Negative',
        'LM_Polarity', 'LM_Subjectivity',
        'NRC_fear', 'NRC_anger', 'NRC_anticip', 'NRC_trust','NRC_surprise', 'NRC_positive', 'NRC_negative', 'NRC_sadness','NRC_disgust', 'NRC_joy', 'NRC_anticipation'
    ]
].groupby(['HOTEL_NAME','REVIEW_YEAR']).mean()

# Save data
df_d3_lg.to_csv("../data/sentiment/sentiment_hy_d3.csv")